#Train The Model On IBM
#TEAM ID : PNT2022TMID38424

### Import necessary libraries

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import keras
from keras.models import Sequential
from keras.layers import Conv2D, Lambda, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization

from keras.preprocessing.image import ImageDataGenerator

from keras.utils.np_utils import to_categorical

from keras.datasets import mnist

### Load the data

In [2]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
print(X_train.shape)
print(X_test.shape)

11490434/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(10000, 28, 28)


### Data pre-processing

In [3]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [4]:
X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)

In [5]:
Y_train = to_categorical(Y_train)
Y_test = to_categorical(Y_test)

In [6]:
mean = np.mean(X_train)
std = np.std(X_train)

def standardize(x):
    return (x-mean)/std

### Create model

In [7]:
model=Sequential()

model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu", input_shape=(28,28,1)))
model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))
model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())    

model.add(Conv2D(filters=256, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
    
model.add(Flatten())
model.add(Dense(512,activation="relu"))
    
model.add(Dense(10,activation="softmax"))
    
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 12, 12, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 128)         1

### Define training parameters

In [9]:
datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=15,
        zoom_range = 0.01,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=False,
        vertical_flip=False)


train_gen = datagen.flow(X_train, Y_train, batch_size=128)
test_gen = datagen.flow(X_test, Y_test, batch_size=128)

In [10]:
epochs = 10
batch_size = 128
train_steps = X_train.shape[0] // batch_size
valid_steps = X_test.shape[0] // batch_size

es = keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=10,
        verbose=1,
        mode="max",
        restore_best_weights=True,
     )

rp = keras.callbacks.ReduceLROnPlateau(
        monitor="val_accuracy",
        factor=0.2,
        patience=3,
        verbose=1,
        mode="max",
        min_lr=0.00001,
     )

### Train the model

In [11]:
history = model.fit(train_gen, 
                    epochs = epochs, 
                    steps_per_epoch = train_steps,
                    validation_data = test_gen,
                    validation_steps = valid_steps, 
                    callbacks=[es, rp])

Epoch 1/10
468/468 [==============================] - 473s 1s/step - loss: 0.1331 - accuracy: 0.9578 - val_loss: 0.2283 - val_accuracy: 0.9189 - lr: 0.0010
Epoch 2/10
468/468 [==============================] - 457s 977ms/step - loss: 0.0536 - accuracy: 0.9834 - val_loss: 0.0787 - val_accuracy: 0.9773 - lr: 0.0010
Epoch 3/10
468/468 [==============================] - 451s 965ms/step - loss: 0.0448 - accuracy: 0.9858 - val_loss: 0.0433 - val_accuracy: 0.9861 - lr: 0.0010
Epoch 4/10
468/468 [==============================] - 449s 960ms/step - loss: 0.0370 - accuracy: 0.9887 - val_loss: 0.0496 - val_accuracy: 0.9864 - lr: 0.0010
Epoch 5/10
468/468 [==============================] - 458s 978ms/step - loss: 0.0337 - accuracy: 0.9897 - val_loss: 0.0301 - val_accuracy: 0.9908 - lr: 0.0010
Epoch 6/10
468/468 [==============================] - 452s 966ms/step - loss: 0.0313 - accuracy: 0.9904 - val_loss: 0.0426 - val_accuracy: 0.9871 - lr: 0.0010
Epoch 7/10
468/468 [=============================

### Save the model

In [12]:
model.save("mnistCNN.h5")

In [13]:
!tar -zcvf mnistCNN.tgz mnistCNN.h5

mnistCNN.h5


#IBM Deployment

### Install necessary packages

In [14]:
!pip install watson-machine-learning-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 538 kB 7.8 MB/s 
     |████████████████████████████████| 55 kB 4.0 MB/s 
     |████████████████████████████████| 132 kB 58.6 MB/s 
     |████████████████████████████████| 9.9 MB 45.1 MB/s 
     |████████████████████████████████| 79 kB 8.7 MB/s 
     |████████████████████████████████| 140 kB 55.1 MB/s 
     |████████████████████████████████| 956 kB 45.2 MB/s 
     |████████████████████████████████| 135 kB 48.4 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.12.0-py3-none-any.whl size=73931 sha256=2186b9fb79d1eb98cdc0848b31716316923c09ecf558f907486b298c48be2031
  Stored in directory: /root/.cache/pip/wheels/ec/94/29/2b57327cf00664b6614304f7958abd29d77ea0e5bbece2ea57
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.12.0-py3-none-any.whl size=562962 sha256=b9b8669ad21

In [15]:
!pip install ibm_watson_machine_learning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 6.1 MB/s 
     |████████████████████████████████| 51 kB 608 kB/s 
     |████████████████████████████████| 824 kB 34.5 MB/s 
     |████████████████████████████████| 133 kB 57.2 MB/s 
     |████████████████████████████████| 547 kB 49.8 MB/s 
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.7.0-py2.py3-none-any.whl size=72563 sha256=d097eae00d058012552cd84149fb535384b90fa4b1534a14c25ab85fa10cf75d
  Stored in directory: /root/.cache/pip/wheels/47/22/bf/e1154ff0f5de93cc477acd0ca69abfbb8b799c5b28a66b44c2
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.7.0-py2.py3-none-any.whl size=501013 sha256=d252d2137b8b3d9c060be05d00d54b9c3841cfa28306f7fee381e15da0389421
  Stored in directory: /root/.cache/pip/wheels/6c/a2/e4/c16d02f809a3ea998e17cfd02c13369281f3d232aaf5902c19
  Created wheel for ibm-cos-sdk-s3transfer: filename=ibm_cos_

### Connect to IBM Watson Machine Learning instance

In [16]:
from ibm_watson_machine_learning import APIClient

API_KEY = "iQN_jIkmwdqiu8eO2z1ktggcV1UEWaDvSz6kjEtHZ_3G"

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": API_KEY
}

client = APIClient(credentials)

Python 3.7 and 3.8 frameworks are deprecated and will be removed in a future release. Use Python 3.9 framework instead.


In [17]:
client

In [19]:
client.spaces.get_details()

{'resources': [{'entity': {'compute': [{'crn': 'crn:v1:bluemix:public:pm-20:us-south:a/371abd374a97422590c408379311cee6:17f4cfdc-cc40-4b22-af24-c6e85315dc8b::',
      'guid': '17f4cfdc-cc40-4b22-af24-c6e85315dc8b',
      'name': 'Watson Machine Learning-kv',
      'type': 'machine_learning'}],
    'description': '',
    'name': 'Handwritten Digit Recognition',
    'scope': {'bss_account_id': '371abd374a97422590c408379311cee6'},
    'stage': {'production': False},
    'status': {'state': 'active'},
    'storage': {'properties': {'bucket_name': '9c7d26c5-0acb-4a6e-8533-6fd92dd9ed7c',
      'bucket_region': 'us-south',
      'credentials': {'admin': {'access_key_id': '22b2a729de4c482c9846be146086fe61',
        'api_key': 'zwqPvnn1_0HCWP53s7UTKXspxEh6_97A9uz23ezoq8Dp',
        'secret_access_key': 'ab8aa46a881b8e2ce2a6963cb603913fe7e552c5d2a9d172',
        'service_id': 'ServiceId-a7101b60-02fc-41b9-860d-2ccf3925d855'},
       'editor': {'access_key_id': '2dba5bf277764cec9b2a6aec081f5241',

In [20]:
client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  -----------------------------  ------------------------
ID                                    NAME                           CREATED
dcaf2242-ad42-49d4-aab1-e5de81276db4  Handwritten Digit Recognition  2022-11-18T07:34:29.106Z
------------------------------------  -----------------------------  ------------------------


In [21]:
space_uid ="dcaf2242-ad42-49d4-aab1-e5de81276db4"
space_uid

'dcaf2242-ad42-49d4-aab1-e5de81276db4'

In [22]:
client.set.default_space(space_uid)

'SUCCESS'

### Define model specifications for deployment

In [23]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [26]:
software_space_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_space_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [27]:
model_details = client.repository.store_model(model="/content/mnistCNN.tgz",meta_props={
    client.repository.ModelMetaNames.NAME: "CNN",
    client.repository.ModelMetaNames.TYPE: "tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_space_uid
})

In [28]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'software_spec': {'id': '12b83a17-24d8-5082-900f-0ab31fbfd3cb',
   'name': 'runtime-22.1-py3.9'},
  'type': 'tensorflow_2.7'},
 'metadata': {'created_at': '2022-11-18T08:33:02.846Z',
  'id': 'd0785188-1959-4372-af19-426ee4ba463c',
  'modified_at': '2022-11-18T08:33:06.130Z',
  'name': 'CNN',
  'owner': 'IBMid-667000C1ID',
  'resource_key': '1b1307e8-8243-4f20-b0eb-757aafe48a3e',
  'space_id': 'dcaf2242-ad42-49d4-aab1-e5de81276db4'},
 'system': {'warnings': []}}

In [30]:
model_id = client.repository.get_model_id(model_details)
model_id

'd0785188-1959-4372-af19-426ee4ba463c'

### Download the deployed model

In [31]:
client.repository.download(model_id, "mnistCNN.tar.gz")

Successfully saved model content to file: 'mnistCNN.tar.gz'


'/content/mnistCNN.tar.gz'